In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [2]:
pre_fine_tuning = load_model("../TumorSense_app/model.h5")

In [3]:
index = ['glioma','meningioma','normal','adenoma']

### fine tune model

In [4]:

dataset_dir = '../../Datasets/brain/Training'

model = load_model('../TumorSense_app/model.h5')
num_classes = len(index)

x = model.layers[-2].output
x = Dense(1024, activation='relu', name='new_dense_1')(x)
predictions = Dense(num_classes, activation='softmax', name='new_dense_2')(x)

fine_tuned_model = Model(inputs=model.input, outputs=predictions)

for layer in fine_tuned_model.layers[:-5]:
    layer.trainable = False

fine_tuned_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 2870 images belonging to 4 classes.


In [5]:
history = fine_tuned_model.fit(test_generator, epochs=10)

test_loss, test_acc = fine_tuned_model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Epoch 1/10


2024-04-15 16:06:11.987392: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


90/90 [==============================] - 12s 119ms/step - loss: 1.6208 - accuracy: 0.7753
Epoch 2/10
90/90 [==============================] - 10s 116ms/step - loss: 0.5878 - accuracy: 0.8895
Epoch 3/10
90/90 [==============================] - 10s 112ms/step - loss: 0.5506 - accuracy: 0.9254
Epoch 4/10
90/90 [==============================] - 10s 114ms/step - loss: 0.7787 - accuracy: 0.8944
Epoch 5/10
90/90 [==============================] - 10s 112ms/step - loss: 0.2056 - accuracy: 0.9575
Epoch 6/10
90/90 [==============================] - 10s 111ms/step - loss: 0.3445 - accuracy: 0.9693
Epoch 7/10
90/90 [==============================] - 10s 112ms/step - loss: 0.1508 - accuracy: 0.9808
Epoch 8/10
90/90 [==============================] - 10s 112ms/step - loss: 0.1216 - accuracy: 0.9787
Epoch 9/10
90/90 [==============================] - 10s 112ms/step - loss: 0.0459 - accuracy: 0.9885
Epoch 10/10
90/90 [==============================] - 10s 110ms/step - loss: 0.3850 - accuracy: 0.9509


In [6]:
# fine_tuned_model.save('../TumorSense_app/fine_tuned_model.h5')